In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Celiac_Disease"
cohort = "GSE138297"

# Input paths
in_trait_dir = "../../input/GEO/Celiac_Disease"
in_cohort_dir = "../../input/GEO/Celiac_Disease/GSE138297"

# Output paths
out_data_file = "../../output/preprocess/Celiac_Disease/GSE138297.csv"
out_gene_data_file = "../../output/preprocess/Celiac_Disease/gene_data/GSE138297.csv"
out_clinical_data_file = "../../output/preprocess/Celiac_Disease/clinical_data/GSE138297.csv"
json_path = "../../output/preprocess/Celiac_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The host response of IBS patients to allogenic and autologous faecal microbiota transfer"
!Series_summary	"In this randomised placebo-controlled trial, irritable bowel syndrome (IBS) patients were treated with faecal material from a healthy donor (n=8, allogenic FMT) or with their own faecal microbiota (n=8, autologous FMT). The faecal transplant was administered by whole colonoscopy into the caecum (30 g of stool in 150 ml sterile saline). Two weeks before the FMT (baseline) as well as two and eight weeks after the FMT, the participants underwent a sigmoidoscopy, and biopsies were collected at a standardised location (20-25 cm from the anal verge at the crossing with the arteria iliaca communis) from an uncleansed sigmoid. In patients treated with allogenic FMT, predominantly immune response-related genes sets were induced, with the strongest response two weeks after FMT. In patients treated with autologous FMT, predominantly metabolism-related g

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Assess Gene Expression Data Availability
# Based on the background information, this dataset appears to be a microarray analysis
# which typically contains gene expression data
is_gene_available = True

# Step 2: Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Trait (Celiac Disease) - Not directly mentioned in the dataset, it's about IBS patients
trait_row = None  # Celiac Disease data is not available in this IBS dataset

# Age - Available in row 3
age_row = 3

# Gender - Available in row 1
gender_row = 1

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    # Not needed since trait data is not available
    return None

def convert_age(value):
    # Extract age value after the colon and convert to integer
    try:
        if value is None:
            return None
        age_str = value.split(': ')[1] if ': ' in value else value
        return int(age_str)
    except:
        return None

def convert_gender(value):
    # The dataset already encodes gender as 0 for male and 1 for female
    # But we need to reverse this to match our convention (0=female, 1=male)
    try:
        if value is None:
            return None
        gender_str = value.split(': ')[1] if ': ' in value else value
        # Convert to integer and then reverse (1->0, 0->1)
        gender_val = int(gender_str)
        return 1 - gender_val  # Reverse the encoding (1->0 for female, 0->1 for male)
    except:
        return None

# Step 3: Save Metadata
# Trait data is not available since trait_row is None
is_trait_available = trait_row is not None

# Initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction
# Since trait_row is None, we skip this step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Celiac_Disease/GSE138297/GSE138297_series_matrix.txt.gz


Gene data shape: (53617, 45)
First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers in the expression data
# These appear to be numeric identifiers (16650001, etc.) and not standard human gene symbols
# Such numeric identifiers typically need to be mapped to standard gene symbols

# Human gene symbols would typically look like: BRCA1, TP53, IL6, etc.
# The identifiers shown are numeric and don't follow gene symbol patterns

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in gene_annotation that contain probe IDs and gene symbols
# From examining the gene_annotation dataframe:
# - 'ID' column contains numerical identifiers matching those in gene_data index
# - 'gene_assignment' column contains gene symbol information

# 2. Create a mapping dataframe with the ID and gene symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the mapped gene expression data
print(f"Gene expression data shape after mapping: {gene_data.shape}")

# Print the first 10 gene symbols to verify the mapping was successful
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Save the gene expression data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression data shape after mapping: (81076, 45)
First 10 gene symbols after mapping:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Celiac_Disease/gene_data/GSE138297.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since we already determined in Step 2 that trait_row is None, 
# we know that trait data is not available in this dataset
print("Trait data not available, skipping clinical feature extraction and data linking")

# Finalize validation that the dataset is not usable
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,
    is_biased=True,  # Setting as True since trait data is missing
    df=pd.DataFrame(),  # Empty dataframe as we're not producing linked data
    note="This dataset contains gene expression data from IBS patients, not Celiac Disease patients. No relevant trait information available."
)

print(f"Dataset usability status: {'Usable' if is_usable else 'Not usable'}")
print("No linked data file saved as trait data is not available.")

Gene data shape after normalization: (23274, 45)
First 10 normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1',
       'A2ML1-AS2', 'A2MP1', 'A4GALT'],
      dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Celiac_Disease/gene_data/GSE138297.csv
Trait data not available, skipping clinical feature extraction and data linking
Abnormality detected in the cohort: GSE138297. Preprocessing failed.
Dataset usability status: Not usable
No linked data file saved as trait data is not available.
